In [1]:
########################################################################
#    July 22, 2019                                                     #
#    16:50                                                             #
#    Created by: Kunal Gehlot                                          #
########################################################################

In [1]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys
import glob 
import warnings
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv2D, Conv1D, MaxPooling2D, AveragePooling1D
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras import regularizers
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.contrib.tensor_forest.python import tensor_forest
from tensorflow.python.ops import resources
from tqdm import tqdm
from matplotlib.pyplot import specgram
from typing import Tuple
# import scipy.io.wavfile as wav
# from speechpy.feature import mfcc
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [2]:
dataList = os.listdir('D:/RawData')

In [3]:
classLabels = ('Angry', 'Fear', 'Disgust', 'Happy', 'Sad', 'Surprised', 'Neutral')

In [4]:
data = []
labels = []
names = []

In [5]:
for number, path in enumerate(tqdm(dataList)):
    X, sample_rate = librosa.load('D:/RawData/'+path, res_type='kaiser_best',duration=2.5,sr=22050*2,offset=0.5)
    sample_rate = np.array(sample_rate)
    mfccs = librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=39)
    feature = mfccs
    data.append(feature)
    
    if path[6:8] == '01' or path[0:1] == 'n':
        labels.append(6)
    if path[6:8] == '02':
        labels.append(6)
    if path[6:8] == '03' or path[0:1] == 'h':
        labels.append(3)
    if path[6:8] == '04' or path[0:2] == 'sa':
        labels.append(4)
    if path[6:8] == '05' or path[0:1] == 'a':
        labels.append(0)
    if path[6:8] == '06' or path[0:1] == 'f':
        labels.append(1)
    if path[6:8] == '07' or path[0:1] == 'd':
        labels.append(2)
    if path[6:8] == '08' or path[0:2] == 'su':
        labels.append(5)
    
    
    names.append(path)
#     break

100%|██████████| 1920/1920 [04:06<00:00,  7.79it/s]


In [6]:
feature.shape

(39, 216)

In [7]:
DATA = np.zeros((1920,39,216))

In [14]:
for i in tqdm(range(0, 1920)):
    for j in range(0, 39):
        for k in range(0, 216):
#             print(i)
#             print(j)
#             print(k)
            temp = data[i]
            temp2 = temp[j]
            if (k >= len(temp2)):
                continue
            else:
                temp3 = temp2[k]
            DATA[i][j][k] = float(temp3)

100%|██████████| 1920/1920 [00:16<00:00, 113.71it/s]


In [15]:
DATA = np.array(DATA)

In [16]:
DATA.shape

(1920, 39, 216)

In [21]:
labels = np.array(labels)

In [22]:
len(labels)

1920

In [23]:
X_train, X_test, Y_train, Y_test = train_test_split(DATA, labels, test_size=0.3, random_state=42)

In [24]:
X_train = np.array(X_train)

In [25]:
X_test = np.array(X_test)

In [26]:
Y_train = np.array(Y_train)

In [27]:
Y_test = np.array(Y_test)

In [28]:
numLabels = len(classLabels)

In [29]:
Y_train = to_categorical(Y_train)

In [30]:
Y_test = to_categorical(Y_test)

In [31]:
in_shape = X_train[0].shape
X_train = X_train.reshape(X_train.shape[0], in_shape[0], in_shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], in_shape[0], in_shape[1], 1)

In [32]:
in_shape

(39, 216)

In [33]:
model = Sequential()

In [34]:
model.add(Conv2D(8, (13, 13), input_shape=(in_shape[0], in_shape[1], 1)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(8, (13, 13)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 1)))
model.add(Conv2D(8, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(8, (1, 1)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 1)))
model.add(Flatten())
model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [35]:
model.add(Dense(numLabels, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam',
                           metrics=['accuracy'])
print(model.summary(), file=sys.stderr)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 27, 204, 8)        1360      
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 204, 8)        32        
_________________________________________________________________
activation_1 (Activation)    (None, 27, 204, 8)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 192, 8)        10824     
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 192, 8)        32        
_________________________________________________________________
activation_2 (Activation)    (None, 15, 192, 8)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 192, 8)         0         
__________

None


In [36]:
best_acc = 0
epochs = 50

In [37]:
for i in tqdm(range(epochs)):
    p = np.random.permutation(len(X_train))
    X_train = X_train[p]
    Y_train = Y_train[p]
    model.fit(X_train, Y_train, batch_size=32, epochs=1)
    loss, acc = model.evaluate(X_test, Y_test)
    if acc > best_acc:
                best_acc = acc

  0%|          | 0/50 [00:00<?, ?it/s]

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
576/576 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - 0s 577us/step


  2%|▏         | 1/50 [00:05<04:34,  5.60s/it]

Epoch 1/1
576/576 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - 0s 326us/step


  4%|▍         | 2/50 [00:07<03:31,  4.41s/it]

...

 96%|█████████▌| 48/50 [01:22<00:03,  1.64s/it]

Epoch 1/1
576/576 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - 0s 325us/step


 98%|█████████▊| 49/50 [01:24<00:01,  1.64s/it]

Epoch 1/1
576/576 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - 0s 327us/step


100%|██████████| 50/50 [01:26<00:00,  1.64s/it]


In [38]:
model.evaluate(X_test, Y_test)

576/576 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - 0s 329us/step


[0.6234208510981666, 0.8459821475876702]

In [39]:
best_acc

0.8643353283405304

In [40]:
model.save_weights('VoiceEmo.h5')

In [41]:
model.load_weights('VoiceEmo.h5')

In [82]:
file = 'output.wav'

In [83]:
X, sample_rate = librosa.load(file, res_type='kaiser_best',duration=2.5,sr=22050*2,offset=0.5)
sample_rate = np.array(sample_rate)
mfccs = librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=39)
feature = mfccs

In [84]:
feature = feature.reshape(39, 216, 1)

In [85]:
classLabels[np.argmax(model.predict(np.array([feature])))]

'Disgust'